<a href="https://colab.research.google.com/github/andresfelipecs/suena_application_test/blob/master/algotrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries

In [33]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
import tensorflow as tf
import time
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

# Loading data

In [34]:
# Load the data into a Pandas DataFrame
df_auction = pd.read_csv("/content/sample_data/auction_data.csv", sep=';', header=0)

# Set the first column as an index 
df_auction['Date (WET)'] = pd.to_datetime(df_auction['Date (WET)'], format='[%d/%m/%Y %H:%M]')
df_auction['date'] = df_auction['Date (WET)']
df_auction.set_index('Date (WET)', inplace=True)


# Drop the first row
units_auction = df_auction.iloc[0]
print(units_auction)

df_auction = df_auction.drop(df_auction.index[0])


# Show the data 
print(df_auction)
# print(f'INDEX:  {df_auction.index} \n')
# print(f'TYPE:  {df_auction.dtypes} \n')
# print(f'SHAPE:  {df_auction.shape} \n')
print(f"COLUMNS: {df_auction.columns} \n")

# print(f'ROW {df_auction.loc["2021-01-01 00:00:00" ]}')




price_first_auction             GBP/MWh
price_second_auction            GBP/MWh
traded_volume_first_auction          MW
traded_volume_second_auction         MW
price_forecast_first_auction    GBP/MWh
date                                NaT
Name: NaT, dtype: object
                    price_first_auction price_second_auction  \
Date (WET)                                                     
2021-01-01 00:00:00                  68                65.73   
2021-01-01 01:00:00                62.7                64.96   
2021-01-01 02:00:00                  56                60.47   
2021-01-01 03:00:00                50.1                 52.5   
2021-01-01 04:00:00                  50                48.98   
...                                 ...                  ...   
2022-09-12 19:00:00               465.6                437.6   
2022-09-12 20:00:00                 418               391.45   
2022-09-12 21:00:00               358.7               342.29   
2022-09-12 22:00:00            

In [46]:
df_forecast_inputs = pd.read_csv(
            "/content/sample_data/forecast_inputs.csv"
        , sep=';', header=0)

# Set the first column as an index 
df_forecast_inputs['Date (WET)'] = pd.to_datetime(df_forecast_inputs['Date (WET)'], format='[%d/%m/%Y %H:%M]')
df_forecast_inputs.set_index('Date (WET)', inplace=True)

# Drop the first row
units_forecast_inputs = df_forecast_inputs.iloc[0]
print(units_forecast_inputs)

df_forecast_inputs = df_forecast_inputs.drop(df_forecast_inputs.index[0])

print(df_forecast_inputs)



demand_plus_system_margin                  MW
demand                                     MW
within_day_availability                    MW
margin                                     MW
within_day_margin                          MW
long_term_wind                        GBP/MWh
long_term_solar                            MW
long_term_wind_over_demand                  %
long_term_wind_over_margin                  %
long_term_solar_over_demand                 %
long_term_solar_over_margin                 %
margin_over_demand                          %
snsp_forecast                               %
stack_price                           GBP/MWh
within_day_potential_stack_price      GBP/MWh
previous_day_ahead_price              GBP/MWh
previous_continuous_half_hour_vwap    GBP/MWh
inertia_forecast                        GVA.s
Name: NaT, dtype: object
                    demand_plus_system_margin   demand  \
Date (WET)                                               
2021-01-01 00:00:00            

In [47]:
df_system_prices = pd.read_csv(
            "/content/sample_data/system_prices.csv"
        , sep=';', header=0)

# Set the first column as an index 
df_system_prices['Date (WET)'] = pd.to_datetime(df_system_prices['Date (WET)'], format='[%d/%m/%Y %H:%M]')
df_system_prices.set_index('Date (WET)', inplace=True)

# Drop the first row
units_system_prices = df_system_prices.iloc[0]
print(units_system_prices)

df_system_prices = df_system_prices.drop(df_system_prices.index[0])


print(df_system_prices)

forecast_system_price_low     GBP/MWh
forecast_system_price_high    GBP/MWh
system_price                  GBP/MWh
Name: NaT, dtype: object
                    forecast_system_price_low forecast_system_price_high  \
Date (WET)                                                                 
2021-01-01 00:00:00                     48.33                      44.89   
2021-01-01 01:00:00                     54.35                      53.26   
2021-01-01 02:00:00                     32.66                      65.86   
2021-01-01 03:00:00                     41.55                      63.43   
2021-01-01 04:00:00                     26.62                      66.91   
...                                       ...                        ...   
2022-09-12 19:00:00                    1285.8                     444.26   
2022-09-12 20:00:00                   -234.22                     390.59   
2022-09-12 21:00:00                     37.57                     324.03   
2022-09-12 22:00:00      

# Split the data into training and testing sets

In [ ]:
train_auction = df_auction[df_auction.index < '2022-03-01']
train_auction = train_auction.apply(pd.to_numeric, errors='coerce')

test_auction = df_auction[df_auction.index >= '2022-03-01']
test_auction = test_auction.apply(pd.to_numeric, errors='coerce')


print(train_auction)
print(test_auction)
print(train_auction.index.inferred_freq)
print(test_auction.index.inferred_freq)
# output None
# output None 

train_forecast_inputs = df_forecast_inputs[df_forecast_inputs.index < '2022-03-01']
train_forecast_inputs = train_forecast_inputs.apply(pd.to_numeric, errors='coerce')

test_forecast_inputs = df_forecast_inputs[df_forecast_inputs.index >= '2022-03-01']
test_forecast_inputs = test_forecast_inputs.apply(pd.to_numeric, errors='coerce')

train_system_prices = df_system_prices[df_system_prices.index < '2022-03-01']
test_system_prices = df_system_prices[df_system_prices.index >= '2022-03-01']

# Plotting trainning **prices** , **volume** and **forecast inputs**

In [38]:

fig = make_subplots(
            rows=2,
            cols=2,
            shared_xaxes=True,
            vertical_spacing=0.08,
            subplot_titles=("1st Action Prices", "2nd Action Prices", "1st Action Volume", "2nd Action Volume"),
            row_width=[0.2, 0.7],
        )

fig.add_trace(
            go.Scatter(x=train_auction.index, y=train_auction["price_first_auction"], mode="lines", line_shape="spline", name="Train Price First Auction", line=dict(color='#80b1d3')),
            row=1,
            col=1,
        )
fig.add_trace(
            go.Scatter(x=train_auction.index, y=train_auction["price_second_auction"], mode="lines", line_shape="spline", name="Train Price Second Auction", line=dict(color='#fb8072')),
            row=1,
            col=2,
        )

fig.add_trace(
            go.Bar(
                x=train_auction.index,
                y=train_auction["traded_volume_first_auction"],
                showlegend=False,
                marker=dict(
                    opacity=1,
                    line=dict(width=2, color="#ffffb3"),
                ),
            ),
            row=2,
            col=1,
        )

fig.add_trace(
            go.Bar(
                x=train_auction.index,
                y=train_auction["traded_volume_second_auction"],
                showlegend=False,
                marker=dict(
                    opacity=1,
                    line=dict(width=2, color="#ffffb3"),
                ),
            ),
            row=2,
            col=2,
        )

fig.add_trace(
            go.Scatter(
                x=train_auction.index,
                y=train_auction["price_forecast_first_auction"],
                mode="lines",
                name="Price train forecast first auction",
                line=dict(color="#fdb462", width=0.5),
            )
        )
            
fig.update_layout(title='Training Actions Prices and Volumes')
fig.update_yaxes(title='Price', row=1)
fig.update_xaxes(title='Date', row=2)

fig.show()

In [39]:

fig = make_subplots(
            rows=2,
            cols=2,
            shared_xaxes=True,
            vertical_spacing=0.08,
            subplot_titles=("1st Action Prices", "2nd Action Prices", "1st Action Volume", "2nd Action Volume"),
            row_width=[0.2, 0.7],
        )

fig.add_trace(
            go.Scatter(x=train_auction.index, y=train_auction["price_first_auction"], mode="lines", line_shape="spline", name="Train Price First Auction", line=dict(color='#80b1d3')),
            row=1,
            col=1,
        )
fig.add_trace(
            go.Scatter(x=train_auction.index, y=train_auction["price_second_auction"], mode="lines", line_shape="spline", name="Train Price Second Auction", line=dict(color='#fb8072')),
            row=1,
            col=2,
        )

fig.add_trace(
            go.Bar(
                x=train_auction.index,
                y=train_auction["traded_volume_first_auction"],
                showlegend=False,
                marker=dict(
                    opacity=1,
                    line=dict(width=2, color="#ffffb3"),
                ),
            ),
            row=2,
            col=1,
        )

fig.add_trace(
            go.Bar(
                x=train_auction.index,
                y=train_auction["traded_volume_second_auction"],
                showlegend=False,
                marker=dict(
                    opacity=1,
                    line=dict(width=2, color="#ffffb3"),
                ),
            ),
            row=2,
            col=2,
        )

fig.add_trace(
            go.Scatter(
                x=train_auction.index,
                y=train_auction["price_forecast_first_auction"],
                mode="lines",
                name="Price train forecast first auction",
                line=dict(color="#fdb462", width=0.5),
            )
        )
            
fig.update_layout(title='Training Actions Prices and Volumes')
fig.update_yaxes(title='Price', row=1)
fig.update_xaxes(title='Date', row=2)

fig.show()

# Plotting test **prices** , **volume** and **forecast inputs**

In [40]:

fig = make_subplots(
            rows=2,
            cols=2,
            shared_xaxes=True,
            vertical_spacing=0.08,
            subplot_titles=("1st Action Prices", "2nd Action Prices", "1st Action Volume", "2nd Action Volume"),
            row_width=[0.2, 0.7],
        )

fig.add_trace(
            go.Scatter(x=test_auction.index, y=test_auction["price_first_auction"], mode="lines", line_shape="spline",name="Test Price First Auction ", line=dict(color='#80b1d3')),
            row=1,
            col=1,
        )
fig.add_trace(
            go.Scatter(x=test_auction.index, y=test_auction["price_second_auction"], mode="lines", line_shape="spline",name="Test Price Second Auction", line=dict(color='#fb8072')),
            row=1,
            col=2,
        )

fig.add_trace(
            go.Bar(
                x=test_auction.index,
                y=test_auction["traded_volume_first_auction"],
                
                showlegend=False,
                marker=dict(
                    opacity=1,
                    line=dict(width=2, color="#ffffb3"),
                ),
            ),
            row=2,
            col=1,
        )

fig.add_trace(
            go.Bar(
                x=test_auction.index,
                y=test_auction["traded_volume_second_auction"],
                
                showlegend=False,
                marker=dict(
                    opacity=1,
                    line=dict(width=2, color="#ffffb3"),
                ),
            ),
            row=2,
            col=2,
        )

fig.add_trace(
            go.Scatter(
                x=test_auction.index,
                y=test_auction["price_forecast_first_auction"],
                mode="lines",
                name="Price train forecast first auction",
                line=dict(color="#fdb462", width=0.5),
            )
        )

fig.update_layout(title='Test Actions Prices and Volumes', showlegend=True)
fig.update_yaxes(title='Price', row=1)
fig.update_xaxes(title='Date', row=2)

fig.show()

# Check the shapes of the resulting training and testing sets





In [41]:
print("train_auction shape: ", train_auction.shape)
print("test_auction shape: ", test_auction.shape)

print("train_forecast_inputs shape: ", train_forecast_inputs.shape)
print("test_forecast_inputs shape: ", test_forecast_inputs.shape)

print("train_system_prices shape: ", train_system_prices.shape)
print("test_system_prices shape: ", test_system_prices.shape)

train_auction shape:  (10176, 6)
test_auction shape:  (4703, 6)
train_forecast_inputs shape:  (10176, 18)
test_forecast_inputs shape:  (4703, 18)
train_system_prices shape:  (10176, 3)
test_system_prices shape:  (4703, 3)


# time series forecasting approach using the SARIMA (p, d, q) (P, D, Q)m

In [42]:
# Let's select the best ARIMA model for the given time series dataset finding the autocorrelation, 
# the partial autocorrelation, the Augmented Dickey-Fuller, the p-value and log difference



In [ ]:
# ACF and PACF ( Autocorrelation, Partial Autocorrelation)

# plot_pacf(train_auction['data']);
# plot_acf(data['data']);

In [44]:
# # Convert the data in the price_second_auction column to a numerical data type
# # train_auction['price_second_auction'] = train_auction['price_second_auction'].astype(float)
# # test_auction['price_second_auction'] = test_auction['price_second_auction'].astype(float)

# # Convert the price columns to numeric values
# train_auction["price_second_auction"] = pd.to_numeric(train_auction["price_second_auction"], errors="coerce")
# test_auction["price_second_auction"] = pd.to_numeric(test_auction["price_second_auction"], errors="coerce")

# # Ensure that the length of the values is the same for both train and test datasets
# train_auction_values = train_auction["price_second_auction"].values[:10176]
# test_auction_values = test_auction["price_second_auction"].values[:4704]

# # Create a new DataFrame for train and test datasets with the correct length of values and index as date range
# train_auction = pd.DataFrame({"price_second_auction": train_auction_values}, 
#                             index=pd.date_range("2021-01-01 00:00:00", "2022-02-28 23:00:00", freq="H"))

# # test_auction = pd.DataFrame({"price_second_auction": test_auction_values}, 
# #                            index=pd.date_range("2022-03-01 00:00:00", "2022-09-12 23:00:00", freq="H"))

# # Fit the ARIMA model to the training data
# model = sm.tsa.ARIMA(train_auction["price_second_auction"], order=(1,0,1))
# model_fit = model.fit()

# print(model_fit.summary())

# start_date = pd.to_datetime("2022-03-01 00:00:00")
# end_date = pd.to_datetime("2022-09-12 23:00:00")

# # Make predictions for the test data
# forecast = model_fit.predict(start=start_date, end=end_date)
# print(forecast)

# # Plot the actual vs. predicted prices using Plotly
# fig = px.line(x=train_auction.index, y=train_auction["price_second_auction"], line_shape="linear", title="Actual vs. Predicted Prices", labels={"x": "Date", "y": "Price"})
# fig.add_scatter(x=test_auction.index, y=forecast, mode="markers", line_shape="spline", name="Predicted Prices")
# fig.show()




In [45]:
# while True:
#   time.sleep(15)